<a href="https://www.kaggle.com/code/velimirkovacic/mn-0036533917-explanatory-data-analysis-eda?scriptVersionId=183358967" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Exploratory data analysis (EDA)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df_train = pd.read_csv("../input/dapprojekt24-1/train.csv")
df_test = pd.read_csv("../input/dapprojekt24-1/test.csv")


## 1. Dimensionality of the dataset 


In [ ]:
#df_train = pd.read_csv('train.csv')  
#df_test = pd.read_csv("test.csv")

print("Train set")
print("Shape:", df_train.shape)

print("Column names: ", end="")
for c in df_train.columns:
    print(c, end= ", ")

print("\nUnique stocks:", df_train["Symbol"].nunique())

print("\n")

print("Test set")
print("Shape:", df_test.shape)

print("Column names: ", end="")
for c in df_test.columns:
    print(c, end= ", ")

print("\nUnique stocks:", df_test["Symbol"].nunique())


if(set(df_test["Symbol"].unique()) == set(df_train["Symbol"].unique())):
    print("\nUnique stocks are the same among both datasets")


df_train

There are two datasets, a train set and a test set. The train set consists of 501400 rows (samples) and 10 columns (features). The test set consists of 209600 samples and 9 features (missing the target feature). 

The column (feature) names are: 
* Date
* Symbol
* Adj Close
* Close
* High
* Low
* Open
* Volume
* Target
* Id

There are 200 different stocks, denoted by their symbol in the Symbol column. They are the same among both datasets.

## 2. Missing values

In [ ]:
def find_missing_sections(df):
    rows_with_missing_values = df.isnull().sum(axis = 1) > 0

    missing_value_indexes = df[rows_with_missing_values].index

    if (len(missing_value_indexes) == 0):
        return []

    # We find the beginning and and indexes of sections with missing values
    missing_value_limits = [missing_value_indexes[0]]
    for i in range(len(missing_value_indexes[1:])):
        if missing_value_indexes[i] - 1 > missing_value_indexes[i - 1]:
            missing_value_limits += [missing_value_indexes[i - 1], missing_value_indexes[i]]
    missing_value_limits += [missing_value_indexes[-1]] 


    # We transform them into (section beginning, section end) pairs
    missing_value_sections = []
    for i in range(0, len(missing_value_limits), 2):
        missing_value_sections += [(missing_value_limits[i], missing_value_limits[i + 1])]

    return missing_value_sections


In [ ]:
print("Train")

missing_value_sections_train = find_missing_sections(df_train)
print("Sections with missing values: ", len(missing_value_sections_train))
for section in missing_value_sections_train:
    print(section, "length:", section[1] - section[0])


print("Test")

missing_value_sections_test = find_missing_sections(df_test)
print("Sections with missing values: ", len(missing_value_sections_test))
for section in missing_value_sections_test:
    print(section, "length:", section[1] - section[0])

There are 27245 rows with one or more missing values in the train set organized in 28 distinct sections. 
* 500 - 515, Only the Volume value is missing. (Replace)
* 17549 - 18524, a section of 975 days with no values except the date and stock symbol (Erase)
* 50140 - 51252, a section of 1112 days with no values except the date and stock symbol (Erase)
* and so on...

For each section we can either replace the missing values or erase the rows. For the section with 15 missing Volume values it would make sense to just replace the value as it is a rather short section and this will not distort the data too much. For the very long sections which are missing every value we have no other choice than to erase them as replacing them would not give us any useful information in the future analysis of the data.

We will erase all rows which have no values except the date and stock symbol and search for missing values again. We will not modify the test dataset as we will still have to make predictions even for the samples with missing values.


In [ ]:
print("Train")
df_train = df_train.dropna(thresh=8)
missing_value_sections_train = find_missing_sections(df_train)
print("Sections with missing values: ", len(missing_value_sections_train))
for section in missing_value_sections_train:
    print(section, "length:", section[1] - section[0])


We are still left with the section of 16 rows missing the value for Volume. We will solve this by using linear interpolation to bridge the gap between row 499 and row 516.

In [ ]:
missing_begin, missing_end = missing_value_sections_train[0]
df_train.loc[:, "Volume"] = df_train["Volume"].interpolate(method='linear')
result_df = df_train.iloc[range(missing_begin - 1, missing_end + 2)]

In [ ]:
print("Train")
df_train = df_train.dropna(thresh=8)
missing_value_sections_train = find_missing_sections(df_train)
print("Sections with missing values: ", len(missing_value_sections_train))
for section in missing_value_sections_train:
    print(section, "length:", section[1] - section[0])

There are no more rows with missing values.

## 3. Duplicates

In [ ]:
print("Train")
duplicate_rows = df_train.duplicated()
print("Numer of duplicate rows:", len(df_train[duplicate_rows]))
duplicate_date_symbol = df_train.duplicated(subset=["Date", "Symbol"])
print("Numer of duplicate Date-Symbol pairs:", len(df_train[duplicate_date_symbol]))
duplicate_id = df_train.duplicated(subset=["Id"])
print("Numer of duplicate Id-s:", len(df_train[duplicate_id]))

print("Test")
duplicate_rows = df_test.duplicated()
print("Numer of duplicate rows:", len(df_test[duplicate_rows]))
duplicate_date_symbol = df_test.duplicated(subset=["Date", "Symbol"])
print("Numer of duplicate Date-Symbol pairs:", len(df_test[duplicate_date_symbol]))
duplicate_id = df_test.duplicated(subset=["Id"])
print("Numer of duplicate Id-s:", len(df_test[duplicate_id]))

There is no duplicate data.

## 4. Data validation

The following must be true for all samples:
1. Adjusted closing price must be less or equal to the closing price.
2. Low price must be less or equal to the high price.
3. Opening and closing price must be in the interval between the low price and the high price.
4. Volume is an integer.
5. All values are positive.

In terms of equations:

* Adj Close &le; Close
* Low &le; High
* Open ∈ [Low, High]
* Close ∈ [Low, High]
* Volume ∈ $\mathbb{Z}$

In [ ]:
def validate_data(df):
    print("Train")
    adjc_bigger_c = df["Adj Close"] > df["Close"]
    print("Rows with Adj Close > Close:", len(df[adjc_bigger_c]))
    low_high = df["Low"] > df["High"]
    print("Rows with Low > High:", len(df[low_high]))
    open_in = (df["Open"] < df["Low"]) | (df["Open"] > df["High"])
    print("Rows with Open < Low or Open > High:", len(df[open_in]))
    close_in = ( df["Close"] < df["Low"]) | (df["Close"] > df["High"])
    print("Rows with Close < Low or Close > High:", len(df[close_in]))
    negative = (df["Adj Close"] < 0) | (df["Close"] < 0) | (df["Open"] < 0) | (df["Low"] < 0) | (df["High"] < 0)
    print("Rows with negative features:", len(df[negative]))


    print("Are they the same? ", end="")
    print((df[negative].index == df[open_in].index).all() and (df[low_high].index == df[open_in].index).all() and (df[close_in].index == df[open_in].index).all())

    isnt_Z = df["Volume"].apply(lambda x: not x.is_integer())
    print("Rows where Volume isn\'t an integer:", len(df[isnt_Z]))

    return negative, isnt_Z


issue1, issue2 = validate_data(df_train)
df_train[issue1]


In [ ]:
df_train[issue2]

There are 110 samples where the High price is negative and about three times higher than the other associated price values. We suspect that the High price is not random but somehow modified from the original price. We can try dividing it by -3 and seeing if the aforementioned constraints are satisfied.

There are also the 16 samples on which we used linear interpolation to fill the gap, we will simply round these to satify the integer constraint.

In [ ]:
df_train.loc[issue1, "High"] /= (-3)
df_train.loc[issue2, "Volume"] = df_train.loc[issue2, "Volume"].round()

_, _ = validate_data(df_train)


Dividing the High price by -3 worked very well, we also tried using a smaller value to divide by such as -3.1, this yielded unfavorable results as not all rows satisfied the constraints. This proces that -3 is the smallest negative value we can use in order to "fix" the prices.

In [ ]:
_, _ = validate_data(df_test)


There are no problematic rows in the test set.

## 5. Feature distributions

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 10))

axes = axes.flatten()

cols = ['Adj Close', 'Close', 'Open', 'High', 'Low', 'Volume']

for i, col in enumerate(cols):
    axes[i].hist(df_train[col], bins=100)
    axes[i].set_title(col + " histogram")
    axes[i].set_xlabel("Stock price")
    axes[i].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

outliers = df_train["Adj Close"] > 1000
df_train[outliers]

The stock NVR is an outlier as it has a much higher prices than the other stocks. This may not be a problem as it could be solved by good feature extraction. We will visualize the features with a logairthmic scale on the y-axis.

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 10))

axes = axes.flatten()

cols = ['Adj Close', 'Close', 'Open', 'High', 'Low', 'Volume']

for i, col in enumerate(cols):
    axes[i].hist(df_train[col], bins=100)
    axes[i].set_title(col + " histogram")
    axes[i].set_yscale("log")
    axes[i].set_xlabel("Stock price")
    axes[i].set_ylabel("Frequency (logarithmic)")

plt.tight_layout()
plt.show()

## 6. Visualization

We will visualize price momevemnts of stocks MMM, NVR and GL for 50 days. We will show the opening and closing price as well as the high and low prices.

In [ ]:
def visualize_stock(symbol, length):
    stock = df_train["Symbol"] == symbol

    fig, ax = plt.subplots(1, 1, figsize=(14, 6))

    for col in ["Open", "Close", "Low", "High"]:
        ax.plot(df_train[stock]["Date"][:length], df_train[stock][col][:length], label=col)
    ax.set_title("50 day plot of stock " + symbol)
    ax.set_xlabel("Date")
    ax.set_xticklabels(df_train[stock]["Date"][:length], rotation=45)
    ax.set_ylabel("Price")
    ax.legend()

    plt.tight_layout()
    plt.show()

visualize_stock("MMM", 50)
visualize_stock("NVR", 50)
visualize_stock("GL", 50)

## 7. Best stock to buy on 15.4.2017.

The question is what makes a stock better to buy than another one. We defined that a stock is worth buying if its price increases at least 2% in the following 2 months. We can thus say that a stock is better to buy if it has the highest price increase in the following 2 months.

The average daily price is calculated as (High - Low)/2.

There is no data for days 14.4.2017., 15.4.2017. and 16.4.2017. We will use the average price of 13.4.2017. and 17.4.2017. as the price on day 15.4.2017. Thankfully, the price for the day 15.5.2017. is available.

In [ ]:
first_date = (df_train["Date"] >= "2017-04-13") & (df_train["Date"] <= "2017-04-17")
avg_first = df_train[first_date].groupby("Symbol")[["High", "Low"]].mean()
avg_first["Price"] = (avg_first["High"] + avg_first["Low"])/2

avg_second = df_train[df_train["Date"] == "2017-06-15"].groupby("Symbol")[["High", "Low"]].mean()
avg_second["Price"] = (avg_second["High"] + avg_second["Low"])/2

price_change = ((avg_second - avg_first) / avg_first)[["Price"]]
price_change.rename(columns={"Price": "Change"}, inplace=True)


stock = price_change["Change"].idxmax()
increase = price_change["Change"].max()
print("Stock:", stock)
print(f"Increase: {increase:.4f}%")


We can see that the stock with the symbol NVDA, denoting NVIDIA Corp has had the highest increase in price in the following 2 months, an increae of 0.5479%.

## 8. Baseline model

We will first create feature sets that out models will be able to use.

String features (Date and Symbol) will be removed. Id will be removed as it carries no useful information. Adjusted Close is correlated to Close so it will also be removed.

Features will be normalized using min-max scaling.

In [ ]:
X_train = df_train.drop(["Id", "Date", "Symbol", "Target", "Adj Close"], axis=1)
y_train = df_train["Target"]

X_test = df_test.drop(["Id", "Date", "Symbol", "Adj Close"], axis=1)

#X['Date'], _ = pd.factorize(X['Date'])
#X['Symbol'], _ = pd.factorize(X['Symbol'])

scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)


We will use this function to evaluate the models.

In [ ]:
def classification_metrics(model):
    y_pred = model.predict(X_train)
    acc = accuracy_score(y_train, y_pred)
    print("Accuracy:", acc)
    cr = classification_report(y_train, y_pred, zero_division=0)
    print("Classification report:\n", cr)
    cm = confusion_matrix(y_train, y_pred)
    print("Confusion matrix:\n", cm)

### 8.1. Logistic Regression


In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

classification_metrics(model)

### 8.2. XGBoost

In [ ]:
model = GradientBoostingClassifier() 
model.fit(X_train, y_train)

classification_metrics(model)

### 8.3. Random Forest

In [ ]:
model = RandomForestClassifier(n_estimators=128)
model.fit(X_train, y_train)

classification_metrics(model)

As we can see, both Logistic Regression and XGBoost classify (almost) every sample into Target = 1 and have a surprisingly high precission because of a disproportion in the number of samples which Target = 0.

Random Forest, on the other hand, has excellent performance on the train set (likely overfitted) and we will use it to predict the test set.

#### Prediction

We have to fill the missing values in the test set in order for Random Forest to properly work. We will use mean fill.

In [ ]:
X_test = X_test.fillna(X_test.mean())

In [ ]:
y_test = model.predict(X_test)
print(y_test)

#### Submission file

In [ ]:
# create submission format
submission = df_test.loc[:,df_test.columns.isin(('Id', ))]

# add random predictions as 'Predicted' column into submission df
submission.loc[:,'Target'] = y_test

# Save predictions to working directiory - this creates submission file
submission.to_csv("submission.csv", index=None)

submission